In [1]:
import os
import numpy as np
import pickle

data = []
labels = []

for filename in os.listdir('./data_preprocessed_python/'):
    if filename =='.dist':
        continue
    with open('./data_preprocessed_python/'+filename, 'rb') as f:
        x = pickle.load(f, encoding='latin1')
        data.append(x['data'])
        labels.append(x['labels'])
data = np.asarray(data)
labels = np.asarray(labels)

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import emd
import torch
import cv2

if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU
else:
    device = torch.device("cpu")   # Use CPU

In [3]:
# Declare Constants Here

sampling_rate = 128  # Hz
sample_points = 8064 # number of points in each time series

In [4]:
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler

class CustomDataset(Dataset):
    def __init__(self,data,labels,channel):
        total_users , total_videos, total_channels, total_samples = np.shape(data) 
        self.data = np.reshape(data,(total_users*total_videos,total_channels,total_samples))
        self.data = self.data[:,channel,:] # Selecting only one channel
        self.data =np.reshape(self.data,(total_users*total_videos,1,total_samples))
        total_users , total_channels, total_samples = np.shape(self.data)
        self.data = StandardScaler().fit_transform(self.data.reshape(-1,1)).reshape(total_users,1,total_samples)
        self.labels = np.reshape(labels,(total_users,4))
        #self.create_windowed_Dataset()
        self.labels_binary_converter()
        # self.shuffle()

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        data = self.data[idx]
        label = self.labels[idx]
        return data, label

    def create_windowed_Dataset(self):
        new_data = []
        for i in range(sample_points - (2*sampling_rate*16),sample_points,sampling_rate*2):
            new_data.append(self.data[:,:,i:i+2*sampling_rate])
        new_data = np.asarray(new_data)
        new_data = new_data.reshape((1280*16,1,256))
        self.data = new_data
    
    def labels_binary_converter(self):
        arousal = self.labels[:,2]
        arousal_threshold = np.median(arousal)
        new_labels = []
        for i in self.labels:
            new_labels.append( [ 1 if i[1] > arousal_threshold else 0])
        new_labels = np.asarray(new_labels)
        self.labels = new_labels.reshape(1280,1)

    def get_all_channels(self,userId):
        plt.plot(self.data[userId,0,:])
    
    # def shuffle(self):
    #     temp = list(zip(self.data,self.labels))
    #     np.random.shuffle(temp)
    #     self.data, self.labels = zip(*temp)
    #     self.data = np.asarray(self.data)
    #     self.labels = np.asarray(self.labels)




In [5]:
dataset = CustomDataset(data,labels,0)

In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ParallelNet(nn.Module):
    def __init__(self):
        super(ParallelNet, self).__init__()
        self.relu = nn.ReLU()
        self.smallerconv = nn.Conv1d(1, 64, 64, stride=8)
        self.largerconv = nn.Conv1d(1, 64, 496, stride=64)
        self.largerpool = nn.MaxPool1d(6,6)
        self.smallerpool = nn.MaxPool1d(3,3)
        self.basepool = nn.MaxPool1d(2,2)
        self.smallmulticonv = nn.Conv1d(64, 128, 8)
        self.intermediatesmallerconv = nn.Conv1d(128, 128, 8)
        self.largemulticonv = nn.Conv1d(64, 128, 6)
        self.intermediatelargerconv = nn.Conv1d(128, 128, 6)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x1 = self.smallerNet(x)
        x2 = self.largerNet(x)
        x = torch.cat((x1,x2),2)
        return x
    
    def smallerNet(self,x):
        x = self.smallerconv(x)
        x = self.relu(x)
        x = self.largerpool(x)
        x = self.dropout(x)
        x = self.smallmulticonv(x)
        x = self.relu(x)
        x = self.intermediatesmallerconv(x)
        x = self.relu(x)
        x = self.intermediatesmallerconv(x)
        x = self.relu(x)
        x = self.smallerpool(x)
        return x
    
    def largerNet(self,x):
        x = self.largerconv(x)
        x = self.relu(x)
        x = self.smallerpool(x)
        x = self.dropout(x)
        x = self.largemulticonv(x)
        x = self.relu(x)
        x = self.intermediatelargerconv(x)
        x = self.relu(x)
        x = self.intermediatelargerconv(x)
        x = self.relu(x)
        x = self.basepool(x)
        return x

In [7]:
# Pre-Training

class DenseNet(nn.Module):
    def __init__(self):
        super(DenseNet,self).__init__()
        self.Linear1 = nn.Linear(7680,256) #4224 = 128*33
        self.relu = nn.ReLU()
        self.Linear2 = nn.Linear(256,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        x = torch.flatten(x,start_dim=1)
        x = self.Linear1(x)
        x = self.relu(x)
        x = self.Linear2(x)
        x = self.sigmoid(x)
        return x
    
# dataloader = DataLoader(dataset, batch_size = 128, shuffle = True) 
# repNet = ParallelNet()
# preTrainNet = nn.Sequential(repNet,DenseNet())
# torch.manual_seed(1000)
# criterion = nn.BCELoss()
# optimizer = optim.Adam(preTrainNet.parameters(), lr=0.001)
# preTrainNet.to(device)
# out = None
# losses = []
# accuracies = []
# for epoch in range(250):
#     epoch_loss = 0
#     epoch_accuracy = 0
#     n=0
#     for i,(data,labels) in enumerate(dataloader):
#         optimizer.zero_grad()
#         data = data.to(device)
#         labels = labels.to(device)
#         out = preTrainNet.forward(data.float())
#         loss = criterion(out, labels.float())
#         loss.backward() #backprop
#         optimizer.step()
#         accuracy = (out.round()==labels).float().mean()
#         epoch_loss += loss
#         epoch_accuracy += accuracy
#         n+=1
#         #print("Batch: ",i," Loss: ",loss," Accuracy: ",accuracy)
#     losses.append(epoch_loss/n)
#     accuracies.append(epoch_accuracy/n)
#     print("Epoch: ",epoch," Loss: ",epoch_loss/n," Accuracy: ",epoch_accuracy/n)

In [8]:
class AttentionNet(nn.Module):
    def __init__(self):
        super(AttentionNet,self).__init__()
        self.query = nn.Conv1d(128, 128, kernel_size=1)
        self.key = nn.Conv1d(128, 128, kernel_size=1)
        self.value = nn.Conv1d(128, 128, kernel_size=1)
        self.multiheadattention = nn.MultiheadAttention(60,1, batch_first = True)
        self.dense1 = nn.Linear(7680,512)
        self.dense2 = nn.Linear(512,64)
        self.dense3 = nn.Linear(64,1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
    def forward(self,x):
        query = self.query(x)
        key = self.key(x)
        value = self.value(x)
        att_out, att_out_weights = self.multiheadattention(query,key,value)
        x = x + att_out
        x = torch.flatten(x,start_dim=1)
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = nn.Sigmoid()(x)
        return x

In [25]:
from sklearn.model_selection import KFold

k_folds = 12
batch_size=128
kf = KFold(n_splits=k_folds, shuffle=True)
models = []
kfold_losses = []
kfold_accuracies = []
l=[]
a=[]
for fold,(train_idx, test_idx) in enumerate(kf.split(dataset)):
    print('-------------------------')
    print('Currently Pre-Training')
    print('-------------------------')
    repNet = ParallelNet()
    preTrainNet = nn.Sequential(repNet,DenseNet())
    torch.manual_seed(1000)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(preTrainNet.parameters(), lr=0.001)
    preTrainNet.to(device)
    train_loader = DataLoader(dataset,batch_size=batch_size,sampler=torch.utils.data.SubsetRandomSampler(train_idx))
    test_loader = DataLoader(dataset,batch_size=batch_size,sampler=torch.utils.data.SubsetRandomSampler(test_idx))
    out = None
    losses = []
    accuracies = []
    for epoch in range(250):
        epoch_loss = 0
        epoch_accuracy = 0
        n=0
        for i,(data,labels) in enumerate(train_loader):
            optimizer.zero_grad()
            data = data.to(device)
            labels = labels.to(device)
            out = preTrainNet.forward(data.float())
            loss = criterion(out, labels.float())
            loss.backward() #backprop
            optimizer.step()
            accuracy = (out.round()==labels).float().mean()
            epoch_loss += loss
            epoch_accuracy += accuracy
            n+=1
    print('-------------------------')
    print(f"Fold : {fold+1}")
    print('-------------------------')
    model = nn.Sequential(repNet,AttentionNet())
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)
    losses = []
    accuracies = []
    models.append(model)
    for epoch in range(101):
        epoch_loss = 0
        epoch_accuracy = 0
        n = 0
        for i,(data,labels) in enumerate(train_loader):
            optimizer.zero_grad()
            data = data.to(device)
            labels = labels.to(device)
            labels = torch.tensor(labels)
            out = model.forward(data.float())
            loss = criterion(out,labels.float())
            loss.backward()
            optimizer.step()
            accuracy = (out.round()==labels).float().mean()
            n+=1
            epoch_loss +=loss
            epoch_accuracy +=accuracy
        losses.append(epoch_loss/n)
        accuracies.append(epoch_accuracy/n)
        print("Epoch: ",epoch," Loss: ",epoch_loss/n," Accuracy: ",epoch_accuracy/n)
    l.append(sum(losses)/len(losses))
    a.append(sum(accuracies)/len(accuracies))
    print('-------------------------')
    print(f'Average Loss: {l[-1]}, Average Accuracy: {a[-1]}')
    print('-------------------------')
    print('Validation Evaluation')
    print('-------------------------')
    model.eval()
    test_loss = 0
    test_accuracy = 0
    with torch.no_grad():
        n=0
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            target = torch.tensor(target)
            out = model(data.float())
            test_loss += criterion(out, target.float())
            test_accuracy += (out.round()==target).float().mean()
            n+=1
    print(f'Loss: {test_loss/n}, Accuracy: {test_accuracy/n}')
    kfold_losses.append(test_loss/n)
    kfold_accuracies.append(test_accuracy/n)
# out

-------------------------
Currently Pre-Training
-------------------------
-------------------------
Validation Evaluation
-------------------------
Loss: 0.25168728828430176, Accuracy: 0.9439252018928528
-------------------------
Currently Pre-Training
-------------------------


C:\Users\Prafful\AppData\Local\Temp\ipykernel_13808\1303967278.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target)


-------------------------
Validation Evaluation
-------------------------
Loss: 0.1697213053703308, Accuracy: 0.9532709717750549
-------------------------
Currently Pre-Training
-------------------------
-------------------------
Validation Evaluation
-------------------------
Loss: 1.325606346130371, Accuracy: 0.9158878326416016
-------------------------
Currently Pre-Training
-------------------------
-------------------------
Validation Evaluation
-------------------------
Loss: 1.2949833869934082, Accuracy: 0.9252336025238037
-------------------------
Currently Pre-Training
-------------------------
-------------------------
Validation Evaluation
-------------------------
Loss: 0.09183298796415329, Accuracy: 0.971962571144104
-------------------------
Currently Pre-Training
-------------------------
-------------------------
Validation Evaluation
-------------------------
Loss: 0.07281015813350677, Accuracy: 0.9813084006309509
-------------------------
Currently Pre-Training
------

In [26]:
kfold_accuracies

[tensor(0.9439, device='cuda:0'),
 tensor(0.9533, device='cuda:0'),
 tensor(0.9159, device='cuda:0'),
 tensor(0.9252, device='cuda:0'),
 tensor(0.9720, device='cuda:0'),
 tensor(0.9813, device='cuda:0'),
 tensor(0.9533, device='cuda:0'),
 tensor(0.9626, device='cuda:0'),
 tensor(0.9340, device='cuda:0'),
 tensor(0.9717, device='cuda:0'),
 tensor(0.9623, device='cuda:0'),
 tensor(0.9906, device='cuda:0')]

In [27]:
torch.mean(torch.tensor(kfold_accuracies))

tensor(0.9555)

In [24]:
model

Sequential(
  (0): ParallelNet(
    (relu): ReLU()
    (smallerconv): Conv1d(1, 64, kernel_size=(64,), stride=(8,))
    (largerconv): Conv1d(1, 64, kernel_size=(496,), stride=(64,))
    (largerpool): MaxPool1d(kernel_size=6, stride=6, padding=0, dilation=1, ceil_mode=False)
    (smallerpool): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (basepool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (smallmulticonv): Conv1d(64, 128, kernel_size=(8,), stride=(1,))
    (intermediatesmallerconv): Conv1d(128, 128, kernel_size=(8,), stride=(1,))
    (largemulticonv): Conv1d(64, 128, kernel_size=(6,), stride=(1,))
    (intermediatelargerconv): Conv1d(128, 128, kernel_size=(6,), stride=(1,))
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (1): AttentionNet(
    (query): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
    (key): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
    (value): Conv1d(128, 128, kernel_size=(1,), stride=

Accuracies using Transformers
[(0.8224,
 0.8692,
 0.8692,
 0.9439,
 0.8224,
 0.9159,
 0.8131,
 0.8224,
 0.9434,
 0.8302,
 0.8774,
 0.8302]

Only the Convolutional Block
[0.9065,
 0.9159,
 0.9346,
 0.9626,
 0.9439,
 0.9720,
 0.9813,
 0.9720,
 0.9811,
 0.9717,
 0.9528,
 0.9717]
